# Resampling Datetime Data (Core)


*Christina Brockway*

Use weather data from London from Kaggle:  https://www.kaggle.com/datasets/emmanuelfwerr/london-weather-data

A modified version for this project:  https://docs.google.com/spreadsheets/d/1J2hEGA1-ZOdXOc1Go2AjB7xShq-nAvIDpBz_XRZscHU/edit?usp=sharing


## Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticks

import missingno as msno

import seaborn as sns
import numpy as np

import datetime as dt



In [2]:
sns.set_context('notebook', font_scale=0.9)
plt.style.use(['ggplot'])

## Load Data

In [3]:

df=pd.read_csv('data/london_weather_MODIFIED - london_weather_MODIFIED.csv')
df.head()

,date,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth
0,19790101,2.0,7.0,52.0,2.3,-4.1,-7.5,0.4,101900.0,9.0
1,19790102,6.0,1.7,27.0,1.6,-2.6,-7.5,0.0,102530.0,8.0
2,19790103,5.0,0.0,13.0,1.3,-2.8,-7.2,0.0,102050.0,4.0
3,19790104,8.0,0.0,13.0,-0.3,-2.6,-6.5,0.0,100840.0,2.0
4,19790105,6.0,2.0,29.0,5.6,-0.8,-1.4,0.0,102250.0,1.0


## Convert to datetime dtype

In [4]:
df['date'] = pd.to_datetime(df['date']+' '+df['Time'],
                            format=fmt,
                            infer_datetime_format=False)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('int64'), dtype('<U1')) -> None

In [ ]:
%%time
## Make a datetime index using pd.to_datetime & specifying the format  (to save gtime)
sp500_min_df['Datetime'] = pd.to_datetime(sp500_min_df['Date']+' ' + sp500_min_df['Time'],
                                          format=fmt,
                                         infer_datetime_format=False)
sp500_min_df